In [136]:
'''
Stellar Evolution Predictor
(Using Mass, Metallicity, and Monte Carlo Uncertainty)

This notebook predict's a star's final evolutionary remnant using astrophysical approximations:

Inputs (from user):
- initial mass in solar masses
- metallicity (Z)

Outputs:
- Main-sequence lifetime
- Effective final mass (M_eff) after mass loss
- Remnant classification
- Monte Carlo probability distribution of final outcomes
- Mass loss distribution (Bar chart)
- Nucleosynthetic yield visualization (Bar chart)

'''

"\nStellar Evolution Predictor\n(Using Mass, Metallicity, and Monte Carlo Uncertainty)\n\nThis notebook predict's a star's final evolutionary remnant using astrophysical approximations:\n\nInputs (from user):\n- initial mass in solar masses\n- metallicity (Z)\n\nOutputs:\n- Main-sequence lifetime\n- Effective final mass (M_eff) after mass loss\n- Remnant classification\n- Monte Carlo probability distribution of final outcomes\n- Mass loss distribution (Bar chart)\n- Nucleosynthetic yield visualization (Bar chart)\n\n"

In [137]:
import numpy as np, matplotlib.pyplot as plt
from collections import Counter
import re
import pandas as pd
from io import StringIO
import glob
import os

In [138]:
# PARSE ISOTOPE YIELD FILES
def parse_yield_file(path):
    ''' parse files with structure:
    H Table: (M=1.0,Z=0.01)
    ...
    Isotopes Yields ... ...
    H-1 ...
    H-2 ...
    Returns: mass, df with isotopes + yields

    fix: finds mass from file name instead
    read online lines that start with Element-Number
    takes the LAST numerical entry to avoid duplicates
    skips malformed/wrapped lines
    '''

    # read file
    with open(path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    # extract mass from filename
    mass = None
    for line in lines:
        if "M=" in line:
            text = line.strip()
            left = text.find("M=") + 2
            right = left
            while right < len(text) and (text[right].isdigit() or text[right] in ".+-eE"):
                right += 1
            mass = float(text[left:right])
            break

    
    isotopes, yields = [], []

    with open(path, errors="ignore") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            
            
            line_parts = line.split()
            isotp_name = line_parts[0]

            if "-" not in isotp_name:
                continue

            y = float(line_parts[1]) #yield will always be second entry no matter the format

            
            isotopes.append(isotp_name)
            yields.append(y)

    df = pd.DataFrame({"Isotopes": isotopes, "Yields": yields})
    df = df.drop_duplicates(subset="Isotopes", keep="first")

    return mass, df


In [139]:
# LOOP TO PARSE ALL FILES

def make_yield_grid(folder):
    dfs, masses = [], []

    for fname in os.listdir(folder):
        if not fname.endswith(".txt"):
            continue
        path = os.path.join(folder, fname)

        mass, df = parse_yield_file(path)
        colname = f"M{mass}"

        df = df.rename(columns={"Yields" : colname})

        dfs.append(df)
        masses.append(mass)

    combined_df = dfs[0]
    for df in dfs[1:]:
        combined_df = combined_df.merge(df, on="Isotopes", how="outer")

    combined_df = combined_df.set_index("Isotopes")

    col_order = sorted(combined_df.columns, key=lambda x: float(x[1:]))
    combined_df = combined_df[col_order]

    return combined_df



In [140]:
df_Z00001 = make_yield_grid("/Users/sarayu/python_decal_fa25/final_project/yield_data/Z00001")
df_Z00002 = make_yield_grid("/Users/sarayu/python_decal_fa25/final_project/yield_data/Z00002")
df_Z0001 = make_yield_grid("/Users/sarayu/python_decal_fa25/final_project/yield_data/Z0001")
df_Z0006 = make_yield_grid("/Users/sarayu/python_decal_fa25/final_project/yield_data/Z0006")
df_Z001 = make_yield_grid("/Users/sarayu/python_decal_fa25/final_project/yield_data/Z001")
df_Z002 = make_yield_grid("/Users/sarayu/python_decal_fa25/final_project/yield_data/Z002")
df_Z003 = make_yield_grid("/Users/sarayu/python_decal_fa25/final_project/yield_data/Z003")

# df_Z0001, df_Z00002, df_Z0001, df_Z0006, df_Z001, df_Z002, df_Z003

df_Z00001

,M1.0,M1.65,M2.0,M3.0,M4.0,M5.0,M6.0,M7.0,M12.0,M15.0,M20.0,M25.0
Isotopes,,,,,,,,,,,,
Ag-107,3.128000e-11,5.785000e-12,3.981000e-12,5.693000e-12,9.420000e-12,5.078000e-12,3.821000e-12,4.428000e-12,7.379000e-12,9.363000e-12,1.145000e-11,1.498000e-11
Ag-109,5.254000e-11,9.694000e-12,6.153000e-12,8.753000e-12,1.501000e-11,6.542000e-12,3.694000e-12,4.259000e-12,6.747000e-12,8.630000e-12,1.093000e-11,1.426000e-11
Al-27,3.506000e-08,3.905000e-07,8.674000e-07,6.366000e-07,2.057000e-06,2.275000e-06,1.841000e-06,1.037000e-06,6.219000e-05,1.157000e-03,1.505000e-03,1.315000e-04
Ar-36,7.335000e-08,1.633000e-07,2.116000e-07,3.492000e-07,5.033000e-07,6.540000e-07,7.965000e-07,9.356000e-07,1.037000e-02,7.309000e-03,2.660000e-03,1.077000e-05
Ar-38,7.857000e-09,1.839000e-08,2.373000e-08,3.471000e-08,4.990000e-08,6.378000e-08,7.707000e-08,9.068000e-08,4.634000e-04,3.752000e-04,8.350000e-04,4.251000e-07
Ar-40,1.071000e-09,3.654000e-10,4.852000e-10,3.740000e-10,5.176000e-10,4.027000e-10,1.762000e-10,1.938000e-10,1.423000e-09,3.986000e-08,1.073000e-08,2.318000e-08
As-75,1.046000e-10,1.708000e-10,1.757000e-10,9.494000e-11,1.224000e-10,9.661000e-11,7.437000e-11,8.289000e-11,5.727000e-10,1.887000e-08,2.289000e-09,1.355000e-09
Au-197,6.544000e-12,1.571000e-12,2.537000e-12,3.316000e-12,7.119000e-12,5.027000e-12,5.224000e-12,6.148000e-12,1.009000e-11,1.274000e-11,1.616000e-11,1.918000e-11
B-11,1.345000e-12,2.949000e-12,3.817000e-12,6.140000e-12,8.848000e-12,1.146000e-11,1.393000e-11,1.637000e-11,2.434000e-11,2.965000e-11,3.004000e-07,3.715000e-08


In [141]:
#INPUT

In [142]:
#MAIN SEQUENCE LIFETIME

In [143]:
#MASS LOSS FUNCTION FROM METALLICITY Z

In [ ]:
# ISOTOPE YIELD INTERPOLATION

'''
For a singular isotope, build a 2d (Z x M) grid
        M1.0        M2.0        ...
Z
0.00001
0.00002
0.001
...

and use this grid to interpolate yield for a specific isotope for any (M, Z)

then, loop through all isotopes to produce a full predicted-yield df
        Yield
Isotopes
H-1        ...
H-2       ...
'''

# combine all metallicites into one dict
yield_grids = {
    0.00001: df_Z00001,
    0.00002: df_Z00002,
    0.0001: df_Z0001,
    0.0006: df_Z0006,
    0.001: df_Z001,
    0.002: df_Z002,
    0.003: df_Z003
}

# get ZxM grid
def isotope_grid(isotope, yield_grids):
    '''
    construct a 2d grid df representing the yields of a SINGLE isotope as a function of Z and M

    parameters:
    isotope: str, isotope name
    yield_grids: dictionary 
    
    returns df with:
    index = Z vals
    cols = sorted mass vals
    vals = yields for the isotope

    '''
    
    Z_vals = sorted(yield_grids.keys()) #sort the metallicities

    #collect all mass column names from any df
    M_cols = sorted([float(c[1:]) for c in list(yield_grids.values())[0].colume])

    grid = []
    for Z in Z_vals:
        df = yield_grids[Z]
        row = df.loc[isotope].values.astype(float) # yields for this isotope at all masses for this Z
        grid.append(row)

    grid_df = pd.DataFrame(grid, index=Z_vals, columns=M_cols)

    grid_df.index.name= "Z"
    grid_df.columns.name= "M"

    return grid_df


# interpolate yield for one isotope at M_targ, Z_targ

def interpolate_isotope(yield_grids, isotope, M_targ, Z_targ):
    ''' returns yield for one isotope at any (M_targ, Z_targ)
    stategy: 2 part interpolation
    1. interpolate along M for each Z
    2. interpolate along Z
    '''    

    Z_vals = sorted(yield_grids.keys())
    yield_atZ = []

    for Z in Z_vals:
        df = yield_grids[Z]
        masses = np.array([float(c[1:]) for c in df.columns])
        values = df.loc[isotope].values.astype(float)

        # interpolate along M for Z
        yield_M = np.interp(M_targ, masses, values)
        yield_atZ.append(yield_M)

    # interpolate along Z
    yield_atZ = np.array(yield_atZ)
    return np.interp(Z_targ, Z_vals, yield_atZ)


# interpolate yield for all isotopes at (M_targ, Z_targ)

def interpolate_all(yield_grids, M_targ, Z_targ):
    ''' returns a df with:
    index = isotopes
    column = yield

    for a set M, Z of the target
    '''

    some_Z = list(yield_grids.keys())[0] # we can get a list of isotopes with any Z table
    isotopes = yield_grids[some_Z].index

    results = []
    for isotope in isotopes:
        val = interpolate_isotope(yield_grids, isotope, M_targ, Z_targ)
        results.append(val)

    df_out = pd.DataFrame({"Yield": results}, index=isotopes)
    df_out.index.name = "Isotopes"
    return df_out



In [ ]:
#testing

df_pred = interpolate_all(yield_grids, M_targ=2.4, Z_targ=0.0015)
df_pred



,Yield
Isotopes,
Ag-107,8.163347e-10
Ag-109,1.274681e-09
Al-27,7.097139e-05
Ar-36,8.675344e-05
Ar-38,1.717010e-05
Ar-40,4.837618e-08
As-75,1.890808e-08
Au-197,1.429050e-09
B-11,3.050388e-09
